In [36]:
import tensorflow.keras as tf
import os
import matplotlib.pyplot as plt
import numpy as np

## Loading in the Data

In [21]:
images = []
for filename in os.listdir("/Users/andrei/code/gan-getting-started/monet_jpg"):
    img = plt.imread(os.path.join("/Users/andrei/code/gan-getting-started/monet_jpg",filename))
    if img is not None:
        images.append(img)

## Normalization

In [29]:
images = np.array(images)

In [30]:
images = images/255

In [31]:
images.shape

(300, 256, 256, 3)

## Building the discriminator

In [41]:
# define the standalone discriminator model
def define_discriminator(in_shape=(256,256,3)):
    model = Sequential()
    model.add(tf.layers.Conv2D(64, (3,3), strides=(2, 2), padding='same', input_shape=in_shape))
    model.add(tf.layers.LeakyReLU(alpha=0.2))
    model.add(tf.layers.Dropout(0.4))
    model.add(tf.layers.Conv2D(64, (3,3), strides=(2, 2), padding='same'))
    model.add(tf.layers.LeakyReLU(alpha=0.2))
    model.add(tf.layers.Dropout(0.4))
    model.add(tf.layers.Flatten())
    model.add(tf.layers.Dense(1, activation='sigmoid'))
    # compile model
    opt = tf.optimizers.Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

## Building the generator

In [42]:
def define_generator(latent_dim):
	model = Sequential()
	# foundation for 7x7 image
	n_nodes = 128 * 64 * 64
	model.add(tf.layers.Dense(n_nodes, input_dim=latent_dim))
	model.add(tf.layers.LeakyReLU(alpha=0.2))
	model.add(tf.layers.Reshape((64, 64, 128)))
	# upsample to 128x128
	model.add(tf.layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(tf.layers.LeakyReLU(alpha=0.2))
	# upsample to 256x256
	model.add(tf.layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(tf.layers.LeakyReLU(alpha=0.2))
	model.add(tf.layers.Conv2D(1, (64,64), activation='sigmoid', padding='same'))
	return model

## The composite model

In [43]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# connect them
	model = tf.Sequential()
	# add generator
	model.add(g_model)
	# add the discriminator
	model.add(d_model)
	# compile model
	opt = tf.Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

In [44]:
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# summarize gan model
gan_model.summary()

/Users/andrei/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


ValueError: Exception encountered when calling layer "sequential_3" (type Sequential).

Input 0 of layer "conv2d_1" is incompatible with the layer: expected axis -1of input shape to have value 3, but received input with shape (None, 256, 256, 1)

Call arguments received:
  • inputs=tf.Tensor(shape=(None, 256, 256, 1), dtype=float32)
  • training=None
  • mask=None